## Compute predictions

Select one of your models and make predictions for the 1,000 unlabeled images from the "cidar4-test.npz" archive. **Explain your choice** in a comment or a markdown cell. **Save the predicted labels** in a "test-predictions.npy" file.

In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
import random

%matplotlib inline

# Load data
with np.load('cifar4-train.npz', allow_pickle=False) as npz_file:
    cifar4_train = dict(npz_file.items())
with np.load('cifar4-test.npz', allow_pickle=False) as npz_file:
    cifar4_test = dict(npz_file.items())

print('Data loaded')

Data loaded


In [3]:
# Create X/y arrays
X_tr = cifar4_train['overfeat']
y_tr = cifar4_train['labels']

X_te = cifar4_test['overfeat']

print('X (train):', X_tr.shape, X_tr.dtype)
print('y (train):', y_tr.shape, y_tr.dtype)

print('X (test):', X_te.shape, X_te.dtype)

X (train): (5000, 4096) float32
y (train): (5000,) int64
X (test): (1000, 4096) float32


In [4]:
# Create the pipeline and fit it to training data
pca = PCA(n_components = None) #the optimum value from the "Logistic regressions" step

logreg = SGDClassifier(
    # Set logistic loss
    loss = 'log',
    alpha = 0.000774263682681, #the optimum value from the "Logistic regressions" step
    # Set max number of iterations and stopping criteria
    max_iter = 1000, tol = 1e-3, random_state = 0
)

# the pipeline considering PCA
pipe_pca = Pipeline([
    ('pca', pca), # The number of components in PCA are adopted from the "Data exploration" section
    ('logreg', logreg)
])

In [7]:
# Fit the tuned pipelines in the entire train set
pipe_pca.fit(X_tr, y_tr)

predictions = np.round(pipe_pca.predict_proba(X_te), 3)

C:\Users\Arash\Anaconda3\envs\exts-aml2\lib\site-packages\sklearn\linear_model\base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [10]:
# Build a DataFrame from predictions
pred_df = pd.DataFrame(data = predictions, columns = ['truck', 'car', 'airplane', 'ship'])

In [28]:
# Find the column name containing the maximum value in each row
pred_labels = pd.DataFrame(data = pred_df.idxmax(axis = 1), columns = ['image label'])
pred_labels

image label
0          ship
1          ship
2      airplane
3          ship
4      airplane
5          ship
6          ship
7          ship
8         truck
9      airplane
10         ship
11        truck
12          car
13          car
14        truck
15     airplane
16         ship
17          car
18     airplane
19        truck
20          car
21        truck
22          car
23     airplane
24         ship
25          car
26     airplane
27        truck
28     airplane
29          car
..          ...
970        ship
971       truck
972        ship
973         car
974       truck
975    airplane
976    airplane
977    airplane
978       truck
979       truck
980        ship
981       truck
982        ship
983         car
984       truck
985        ship
986    airplane
987    airplane
988    airplane
989       truck
990         car
991         car
992        ship
993       truck
994       truck
995         car
996         car
997       truck
998        ship
999        ship

[1000 rows x 1 columns]

In [25]:
# Save the predictions into a .npy file
np.save('test-predictions.npy', pred_labels)

The **Logistic Regressions** model was selected to predict the labels of test values. **Neural Networks** have a tremendous capability to fit the model to the train data, even in the presence of severe nonlinearities in the data. But, they are prone to the presence of outliers in the data as in our case for **OverFeat Features** as seen in the *Data Exploration* step. Moreover, there should be enough data available with respect to the features so that the **Neural Networks** can train the model perfectly. As seen in the results sections, the accuracy of **Neural Network** models decrease combined to the **Logistic Regressions** model. It seems that with the number of available train data, **Logistic Regressions** would yield more accurate results even with its simpler approach.